In [1]:
import ipywidgets as widgets
from ipywidgets import interact
from ipyfilechooser import FileChooser
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import functools
from typing import Callable


# Create file choosers
folder_chooser = FileChooser(use_dir_icons=True, show_only_dirs=True, title='Select working folder:', 
                             layout=widgets.Layout(width='50%'))
train_csv_chooser = FileChooser(use_dir_icons=True, filter_pattern=["*.csv"], title='Select train data (.csv):', 
                              layout=widgets.Layout(width='50%'))
test_csv_chooser = FileChooser(use_dir_icons=True, filter_pattern=["*.csv"], title='Select test data (.csv):', 
                              layout=widgets.Layout(width='50%'))

# Create tab 1 contents
tab1_folder_label = widgets.Label(value='Folder chooser:', layout=widgets.Layout(width='200px'))
tab1_folder_box = widgets.HBox([tab1_folder_label, folder_chooser])
tab1_train_csv_label = widgets.Label(value='Train .csv chooser:', layout=widgets.Layout(width='200px'))
tab1_test_csv_label = widgets.Label(value='Test .csv chooser:', layout=widgets.Layout(width='200px'))
tab1_file_box = widgets.VBox([widgets.HBox([tab1_train_csv_label, train_csv_chooser]), widgets.HBox([tab1_test_csv_label, test_csv_chooser])])

In [2]:
# Create tab 2 contents
# Create dropdowns for selecting columns
tab2_dropdown1 = widgets.Dropdown(options=[], layout=widgets.Layout(width='200px'))
tab2_dropdown2 = widgets.Dropdown(options=[], layout=widgets.Layout(width='200px'))

# Create warning label
warning_label = widgets.Label(value='Warning: you need to select different columns.', layout=widgets.Layout(visibility="hidden", width='50%'))
def callback_dropdown_warning(*args):
    if tab2_dropdown1.value == tab2_dropdown2.value and tab2_dropdown1.value and tab2_dropdown2.value:
        warning_label.layout.visibility = "visible"
    else:
        warning_label.layout.visibility = "hidden"

tab2_dropdown2.observe(callback_dropdown_warning, names="value")
tab2_dropdown1.observe(callback_dropdown_warning, names="value")

# Function to update dropdowns and show/hide warning label
def update_dropdowns(*args):
    if train_csv_chooser.selected:
        df = pd.read_csv(train_csv_chooser.selected)
        tab2_dropdown1.options = list(df.columns)
        tab2_dropdown2.options = list(df.columns)
# Attach the update_dropdowns function to the parquet file chooser
train_csv_chooser.register_callback(update_dropdowns)

tab2_dropdown1_label = widgets.Label(value='Select x-data column:', layout=widgets.Layout(width='200px'))
tab2_dropdown1_box = widgets.HBox([tab2_dropdown1_label, tab2_dropdown1])

tab2_dropdown2_label = widgets.Label(value='Select label (y) column:', layout=widgets.Layout(width='200px'))
tab2_dropdown2_box = widgets.HBox([tab2_dropdown2_label, tab2_dropdown2])

tab2_warning_box = widgets.VBox([warning_label])

In [3]:
tab2_label_render = widgets.Label("Render plot:", layout=widgets.Layout(width='200px'))
tab2_button_render = widgets.Button(description="render", layout=widgets.Layout(width='100px'))
tab2_button_box = widgets.HBox([tab2_label_render, tab2_button_render])

tab2_plot_label = widgets.Label("Label-vs-x plot", layout=widgets.Layout(width='200px'))

import random

def plotly_to_widget(plotly_func: Callable) -> Callable:
    @functools.wraps(plotly_func)
    def wrapper(*args, **kwargs):
        return go.FigureWidget(plotly_func(*args, **kwargs))

    return wrapper

def render_plot(*args):
    if tab2_dropdown1.value != tab2_dropdown2.value and tab2_dropdown1.value and tab2_dropdown2.value:
        x_col = tab2_dropdown1.value
        y_col = tab2_dropdown2.value
        df = pd.read_csv(train_csv_chooser.value)
        fig = go.FigureWidget(go.Scatter(x=df[x_col].to_numpy(), y=df[y_col].to_numpy()))
        
        
        fig = go.Figure()

        fig.add_trace(
            go.Scatter(
                x=list(trace["x_data"]),
                y=list(trace["y_data"]),
                name=trace["name"],
                text=trace["dot_labels"],
                hovertemplate="%{text}",
                legendgroup=trace["group"],
                legendgrouptitle_text=trace["group"].capitalize(),
                mode="markers",
                marker=trace["marker"],
                visible=trace["visible"],
            )
        )
        
        return fig
        
def render_then_box(*args):
    fig = render_plot()
    tab2_plot_box = widgets.HBox([tab2_plot_label, fig])
    display(tab2_plot_box)
        
tab2_button_render.on_click(render_then_box)


In [4]:
panel_widget = widgets.VBox([
    tab1_folder_box, 
    tab1_file_box, 
    tab2_dropdown1_box, 
    tab2_dropdown2_box, 
    tab2_warning_box, 
    tab2_button_box],
)

# Display the tabs widget
display(panel_widget)

    'data': [{'typ…

In [ ]:



#fig.show()

@interact
def render_plot(*args):
    if tab2_dropdown1.value != tab2_dropdown2.value and tab2_dropdown1.value and tab2_dropdown2.value:
        x_col = tab2_dropdown1.value
        y_col = tab2_dropdown2.value
        df = pd.read_csv(train_csv_chooser.value)
        fig = go.FigureWidget()
        fig.add_scatter(x=df[x_col].to_numpy(), y=df[y_col].to_numpy())
        tab2_plot_box = widgets.HBox([tab2_plot_label, fig])
        display(fig)
tab2_button_render.on_click(render_plot)